<a href="https://colab.research.google.com/github/tfindiamooc/tfindiamooc.github.io/blob/master/colabs/tf_keras_overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## tf.keras
We have extensively used tf.Keras API throughout the course. Let's review main concepts behind tf.Keras before learning how to customize it.

tf.keras is TensorFlow's implementation of the Keras API specification. This is a high-level API to build and train models that includes first-class support for TensorFlow-specific functionality, such as eager execution, tf.data pipelines, and Estimators. tf.keras makes TensorFlow easier to use without sacrificing flexibility and performance.


In [0]:
# Note: Select 'GPU' hardware accelerator

from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

# We need to import keras from tensorflow package.
from tensorflow import keras

TensorFlow 2.x selected.


`tf.keras` can run any Keras-compatible code, but keep in mind:

* The `tf.keras` version in the latest TensorFlow release might not be the same
  as the latest `keras` version from PyPI. Check `tf.keras.__version__`.
* When [saving a model's weights](#weights_only), `tf.keras` defaults to the
  [checkpoint format](./checkpoints.md). Pass `save_format='h5'` to
  use HDF5 (or pass a filename that ends in `.h5`).

## Build a simple model

### Sequential model

In Keras, we assemble *layers* to build *models*. A model is (usually) a *graph
of layers*. The most common type of model is a stack of layers: the
`tf.keras.Sequential` model.

To build a simple, fully-connected network (i.e. multi-layer perceptron) as we have seen in numerous occassions in this course:

In [0]:
from tensorflow.keras import layers

model = tf.keras.Sequential()
# Adds a densely-connected layer with 64 units to the model:
model.add(layers.Dense(64, activation='relu'))
# Add another:
model.add(layers.Dense(64, activation='relu'))
# Add a softmax layer with 10 output units:
model.add(layers.Dense(10, activation='softmax'))

We will learn how to learn how to write advance models by - 
* Implementing layers and models from scratch with subclassing.
* Functional APIs

### Configure the layers

There are many `tf.keras.layers` available. Most of them share some common constructor
arguments:

* `activation`: Set the activation function for the layer. This parameter is
  specified by the name of a built-in function or as a callable object. By
  default, no activation is applied.
* `kernel_initializer` and `bias_initializer`: The initialization schemes
  that create the layer's weights (kernel and bias). This parameter is a name or
  a callable object. This defaults to the `"Glorot uniform"` initializer.
* `kernel_regularizer` and `bias_regularizer`: The regularization schemes
  that apply the layer's weights (kernel and bias), such as L1 or L2
  regularization. By default, no regularization is applied.

The following instantiates `tf.keras.layers.Dense` layers using constructor
arguments:

In [0]:
# Create a sigmoid layer:
layers.Dense(64, activation='sigmoid')
# Or:
layers.Dense(64, activation=tf.keras.activations.sigmoid)

# A linear layer with L1 regularization of factor 0.01 applied to the kernel matrix:
layers.Dense(64, kernel_regularizer=tf.keras.regularizers.l1(0.01))

# A linear layer with L2 regularization of factor 0.01 applied to the bias vector:
layers.Dense(64, bias_regularizer=tf.keras.regularizers.l2(0.01))

# A linear layer with a kernel initialized to a random orthogonal matrix:
layers.Dense(64, kernel_initializer='orthogonal')

# A linear layer with a bias vector initialized to 2.0s:
layers.Dense(64, bias_initializer=tf.keras.initializers.Constant(2.0))

## Train and evaluate

### Set up training

After the model is constructed, configure its learning process by calling the
`compile` method:

In [0]:
model = tf.keras.Sequential([
# Adds a densely-connected layer with 64 units to the model:
layers.Dense(64, activation='relu', input_shape=(32,)),
# Add another:
layers.Dense(64, activation='relu'),
# Add a softmax layer with 10 output units:
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

`tf.keras.Model.compile` takes three important arguments:

* `optimizer`: This object specifies the training procedure. Pass it optimizer
  instances from the `tf.keras.optimizers` module, such as
  `tf.keras.optimizers.Adam` or
  `tf.keras.optimizers.SGD`. If you just want to use the default parameters, you can also specify optimizers via strings, such as `'adam'` or `'sgd'`.
* `loss`: The function to minimize during optimization. Common choices include
  mean square error (`mse`), `categorical_crossentropy`, and
  `binary_crossentropy`. Loss functions are specified by name or by
  passing a callable object from the `tf.keras.losses` module.
* `metrics`: Used to monitor training. These are string names or callables from
  the `tf.keras.metrics` module.
* Additionally, to make sure the model trains and evaluates eagerly, you can make sure to pass `run_eagerly=True` as a parameter to compile.


The following shows a few examples of configuring a model for training:

In [0]:
# Configure a model for mean-squared error regression.
model.compile(optimizer=tf.keras.optimizers.Adam(0.01),
              loss='mse',       # mean squared error
              metrics=['mae'])  # mean absolute error

# Configure a model for categorical classification.
model.compile(optimizer=tf.keras.optimizers.RMSprop(0.01),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=[tf.keras.metrics.CategoricalAccuracy()])

### Train from NumPy data

For small datasets, use in-memory [NumPy](https://www.numpy.org/)
arrays to train and evaluate a model. The model is "fit" to the training data
using the `fit` method:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
print ("Data:")
print (data.shape)
print (data)
print ()

labels = np.random.random((1000, 10))
print ("Labels:")
print (labels.shape)
print (labels)
print ()


model.fit(data, labels, epochs=10, batch_size=32)

Data:
(1000, 32)
[[0.46279467 0.54542336 0.33083747 ... 0.61531252 0.55198904 0.20527891]
 [0.02228185 0.01718279 0.07512161 ... 0.63011734 0.23376771 0.11686902]
 [0.20154362 0.95562986 0.61506725 ... 0.34543211 0.83386872 0.11255925]
 ...
 [0.38310815 0.08241507 0.61976597 ... 0.77570641 0.89220555 0.62323299]
 [0.89114165 0.40249842 0.98120854 ... 0.20277798 0.47728665 0.1823137 ]
 [0.11069933 0.38474998 0.69744117 ... 0.89963302 0.72440524 0.97446799]]

Labels:
(1000, 10)
[[0.76904526 0.66029964 0.93793141 ... 0.97695328 0.94925192 0.47278172]
 [0.01163332 0.590032   0.34382257 ... 0.88194098 0.11518792 0.91361061]
 [0.0372053  0.90358955 0.80539344 ... 0.2465504  0.41983513 0.53473651]
 ...
 [0.88315112 0.14233738 0.28011601 ... 0.73798099 0.21078974 0.64289613]
 [0.51889558 0.44070195 0.2204166  ... 0.50133808 0.40221377 0.13385512]
 [0.38901634 0.48716222 0.49771212 ... 0.61110588 0.2839987  0.16715763]]

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================

`tf.keras.Model.fit` takes three important arguments:

* `epochs`: Training is structured into *epochs*. An epoch is one iteration over
  the entire input data (this is done in smaller batches).
* `batch_size`: When passed NumPy data, the model slices the data into smaller
  batches and iterates over these batches during training. This integer
  specifies the size of each batch. Be aware that the last batch may be smaller
  if the total number of samples is not divisible by the batch size.
* `validation_data`: When prototyping a model, you want to easily monitor its
  performance on some validation data. Passing this argument—a tuple of inputs
  and labels—allows the model to display the loss and metrics in inference mode
  for the passed data, at the end of each epoch.

Here's an example using `validation_data`:

In [0]:
import numpy as np

data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

val_data = np.random.random((100, 32))
val_labels = np.random.random((100, 10))

model.fit(data, labels, epochs=10, batch_size=32,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/10
1000/1000 [==============================] - 0s 185us/sample - loss: 20713.8682 - categorical_accuracy: 0.1030 - val_loss: 27396.7330 - val_categorical_accuracy: 0.1100
Epoch 2/10
1000/1000 [==============================] - 0s 118us/sample - loss: 25402.6138 - categorical_accuracy: 0.1010 - val_loss: 22097.6877 - val_categorical_accuracy: 0.1600
Epoch 3/10
1000/1000 [==============================] - 0s 118us/sample - loss: 28807.4465 - categorical_accuracy: 0.0930 - val_loss: 29128.5859 - val_categorical_accuracy: 0.1300
Epoch 4/10
1000/1000 [==============================] - 0s 106us/sample - loss: 32337.9998 - categorical_accuracy: 0.0900 - val_loss: 40753.5725 - val_categorical_accuracy: 0.1100
Epoch 5/10
1000/1000 [==============================] - 0s 113us/sample - loss: 36592.3249 - categorical_accuracy: 0.1020 - val_loss: 36947.6856 - val_categorical_accuracy: 0.0600
Epoch 6/10
1000/1000 [==============================]

### Train from tf.data datasets

Use the [Datasets API](../data.md) to scale to large datasets
or multi-device training. Pass a `tf.data.Dataset` instance to the `fit`
method:

In [0]:
# Instantiates a toy dataset instance:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.fit(dataset, epochs=10)

Epoch 1/10
32/32 [==============================] - 0s 6ms/step - loss: 70293.3820 - categorical_accuracy: 0.1040
Epoch 2/10
32/32 [==============================] - 0s 3ms/step - loss: 77965.0060 - categorical_accuracy: 0.0870
Epoch 3/10
32/32 [==============================] - 0s 3ms/step - loss: 84647.7776 - categorical_accuracy: 0.1060
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 92035.7500 - categorical_accuracy: 0.0980
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 99148.3030 - categorical_accuracy: 0.0900
Epoch 6/10
32/32 [==============================] - 0s 4ms/step - loss: 107681.4515 - categorical_accuracy: 0.0970
Epoch 7/10
32/32 [==============================] - 0s 3ms/step - loss: 112926.8245 - categorical_accuracy: 0.0890
Epoch 8/10
32/32 [==============================] - 0s 3ms/step - loss: 122278.9195 - categorical_accuracy: 0.1000
Epoch 9/10
32/32 [==============================] - 0s 3ms/step - loss: 130185.9002 -

Since the `Dataset` yields batches of data, this snippet does not require a `batch_size`.

Datasets can also be used for validation:

In [0]:
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((val_data, val_labels))
val_dataset = val_dataset.batch(32)

model.fit(dataset, epochs=10,
          validation_data=val_dataset)

Epoch 1/10
32/32 [==============================] - 0s 6ms/step - loss: 150477.4675 - categorical_accuracy: 0.0880 - val_loss: 0.0000e+00 - val_categorical_accuracy: 0.0000e+00
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 155405.9561 - categorical_accuracy: 0.1000 - val_loss: 193615.5391 - val_categorical_accuracy: 0.1100
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 173290.5112 - categorical_accuracy: 0.0990 - val_loss: 174782.0469 - val_categorical_accuracy: 0.0900
Epoch 4/10
32/32 [==============================] - 0s 3ms/step - loss: 177458.3169 - categorical_accuracy: 0.1010 - val_loss: 154702.8594 - val_categorical_accuracy: 0.1300
Epoch 5/10
32/32 [==============================] - 0s 3ms/step - loss: 186760.2845 - categorical_accuracy: 0.0890 - val_loss: 132763.9062 - val_categorical_accuracy: 0.1300
Epoch 6/10
32/32 [==============================] - 0s 3ms/step - loss: 202978.8757 - categorical_accuracy: 0.0810 - val_loss: 

### Evaluate and predict

The `tf.keras.Model.evaluate` and `tf.keras.Model.predict` methods can use NumPy
data and a `tf.data.Dataset`.

Here's how to *evaluate* the inference-mode loss and metrics for the data provided:

In [0]:
# With Numpy arrays
data = np.random.random((1000, 32))
labels = np.random.random((1000, 10))

model.evaluate(data, labels, batch_size=32)

# With a Dataset
dataset = tf.data.Dataset.from_tensor_slices((data, labels))
dataset = dataset.batch(32)

model.evaluate(dataset)

1000/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

[212249.6767578125, 0.103]

And here's how to *predict* the output of the last layer in inference for the data provided,
as a NumPy array:

In [0]:
result = model.predict(data, batch_size=32)
print(result.shape)

(1000, 10)


## Callbacks

A callback is an object passed to a model to customize and extend its behavior
during training. You can write your own custom callback, or use the built-in
`tf.keras.callbacks` that include:

* `tf.keras.callbacks.ModelCheckpoint`: Save checkpoints of your model at
  regular intervals.
* `tf.keras.callbacks.LearningRateScheduler`: Dynamically change the learning
  rate.
* `tf.keras.callbacks.EarlyStopping`: Interrupt training when validation
  performance has stopped improving.
* `tf.keras.callbacks.TensorBoard`: Monitor the model's behavior using
  [TensorBoard](https://tensorflow.org/tensorboard).

To use a `tf.keras.callbacks.Callback`, pass it to the model's `fit` method:

In [0]:
callbacks = [
  # Interrupt training if `val_loss` stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
  # Write TensorBoard logs to `./logs` directory
  tf.keras.callbacks.TensorBoard(log_dir='./logs')
]
model.fit(data, labels, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(val_data, val_labels))

Train on 1000 samples, validate on 100 samples
Epoch 1/5
1000/1000 [==============================] - 0s 375us/sample - loss: 254400.4095 - categorical_accuracy: 0.0960 - val_loss: 223977.8494 - val_categorical_accuracy: 0.0600
Epoch 2/5
1000/1000 [==============================] - 0s 112us/sample - loss: 263147.2252 - categorical_accuracy: 0.1080 - val_loss: 243316.6650 - val_categorical_accuracy: 0.1300
Epoch 3/5
1000/1000 [==============================] - 0s 114us/sample - loss: 275122.5640 - categorical_accuracy: 0.0930 - val_loss: 255566.6375 - val_categorical_accuracy: 0.1600


## Save and restore

### Save just the weights values

Save and load the weights of a model using `tf.keras.Model.save_weights`:

In [0]:
model = tf.keras.Sequential([
layers.Dense(64, activation='relu', input_shape=(32,)),
layers.Dense(10, activation='softmax')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# Save weights to a TensorFlow Checkpoint file
model.save_weights('./weights/my_model')

# Restore the model's state,
# this requires a model with the same architecture.
model.load_weights('./weights/my_model')

In [0]:
!ls

logs  my_model.h5  sample_data	weights


In [0]:
!ls weights

checkpoint		      my_model.data-00001-of-00002
my_model.data-00000-of-00002  my_model.index


By default, this saves the model's weights in the
[TensorFlow checkpoint](../checkpoints.md) file format. Weights can
also be saved to the Keras HDF5 format (the default for the multi-backend
implementation of Keras):

In [0]:
# Save weights to a HDF5 file
model.save_weights('my_model.h5', save_format='h5')

# Restore the model's state
model.load_weights('my_model.h5')

In [0]:
!ls

logs  my_model.h5  sample_data	weights


### Save just the model configuration

A model's configuration can be saved—this serializes the model architecture
without any weights. A saved configuration can recreate and initialize the same
model, even without the code that defined the original model. Keras supports
JSON and YAML serialization formats:

In [0]:
# Serialize a model to JSON format
json_string = model.to_json()
json_string

'{"class_name": "Sequential", "config": {"name": "sequential_2", "layers": [{"class_name": "Dense", "config": {"name": "dense_12", "trainable": true, "batch_input_shape": [null, 32], "dtype": "float32", "units": 64, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_13", "trainable": true, "dtype": "float32", "units": 10, "activation": "softmax", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}]}, "keras_version": "2.2.4-tf", "backen

In [0]:
import json
import pprint
pprint.pprint(json.loads(json_string))

{'backend': 'tensorflow',
 'class_name': 'Sequential',
 'config': {'layers': [{'class_name': 'Dense',
                        'config': {'activation': 'relu',
                                   'activity_regularizer': None,
                                   'batch_input_shape': [None, 32],
                                   'bias_constraint': None,
                                   'bias_initializer': {'class_name': 'Zeros',
                                                        'config': {}},
                                   'bias_regularizer': None,
                                   'dtype': 'float32',
                                   'kernel_constraint': None,
                                   'kernel_initializer': {'class_name': 'GlorotUniform',
                                                          'config': {'seed': None}},
                                   'kernel_regularizer': None,
                                   'name': 'dense_12',
                            


Recreate the model (newly initialized) from the JSON:

In [0]:
fresh_model = tf.keras.models.model_from_json(json_string)

Serializing a model to YAML format requires that you install `pyyaml` *before you import TensorFlow*:

In [0]:
yaml_string = model.to_yaml()
print(yaml_string)

backend: tensorflow
class_name: Sequential
config:
  layers:
  - class_name: Dense
    config:
      activation: relu
      activity_regularizer: null
      batch_input_shape: !!python/tuple [null, 32]
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_12
      trainable: true
      units: 64
      use_bias: true
  - class_name: Dense
    config:
      activation: softmax
      activity_regularizer: null
      bias_constraint: null
      bias_initializer:
        class_name: Zeros
        config: {}
      bias_regularizer: null
      dtype: float32
      kernel_constraint: null
      kernel_initializer:
        class_name: GlorotUniform
        config: {seed: null}
      kernel_regularizer: null
      name: dense_13
      trai

Recreate the model from the YAML:

In [0]:
fresh_model = tf.keras.models.model_from_yaml(yaml_string)

Note: Subclassed models are not serializable because their architecture is defined by the Python code in the body of the call method.

### Save the entire model in one file

The entire model can be saved to a file that contains the weight values, the
model's configuration, and even the optimizer's configuration. This allows you
to checkpoint a model and resume training later—from the exact same
state—without access to the original code.

In [0]:
# Create a simple model
model = tf.keras.Sequential([
  layers.Dense(10, activation='softmax', input_shape=(32,)),
  layers.Dense(10, activation='softmax')
])
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(data, labels, batch_size=32, epochs=5)


# Save entire model to a HDF5 file
model.save('my_model.h5')

# Recreate the exact same model, including weights and optimizer.
model = tf.keras.models.load_model('my_model.h5')

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 1s 508us/sample - loss: 11.4986 - accuracy: 0.1040
Epoch 2/5
1000/1000 [==============================] - 0s 90us/sample - loss: 11.5157 - accuracy: 0.1080
Epoch 3/5
1000/1000 [==============================] - 0s 98us/sample - loss: 11.5469 - accuracy: 0.1130
Epoch 4/5
1000/1000 [==============================] - 0s 100us/sample - loss: 11.5803 - accuracy: 0.1090
Epoch 5/5
1000/1000 [==============================] - 0s 102us/sample - loss: 11.6106 - accuracy: 0.1090
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


### Multiple GPUs

`tf.keras` models can run on multiple GPUs using
`tf.distribute.Strategy`. This API provides distributed
training on multiple GPUs with almost no changes to existing code.

Currently, `tf.distribute.MirroredStrategy` is the only supported
distribution strategy. `MirroredStrategy` does in-graph replication with
synchronous training using all-reduce on a single machine. To use
`distribute.Strategy`s , nest the optimizer instantiation and model construction and compilation in a `Strategy`'s `.scope()`, then
train the model.

The following example distributes a `tf.keras.Model` across multiple GPUs on a
single machine.

First, define a model inside the distributed strategy scope:

In [0]:
strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
  model = tf.keras.Sequential()
  model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
  model.add(layers.Dense(1, activation='sigmoid'))

  optimizer = tf.keras.optimizers.SGD(0.2)

  model.compile(loss='binary_crossentropy', optimizer=optimizer)

model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 17        
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________


Next, train the model on data as usual:

In [0]:
x = np.random.random((1024, 10))
y = np.random.randint(2, size=(1024, 1))
x = tf.cast(x, tf.float32)
dataset = tf.data.Dataset.from_tensor_slices((x, y))
dataset = dataset.shuffle(buffer_size=1024).batch(32)

model.fit(dataset, epochs=1)

32/32 [==============================] - 3s 78ms/step - loss: 0.7099
